In [38]:
import requests
import json
import random 

URL = "http://localhost:"
ports = range(10000,10009)
def random_url():
    return (URL + str(random.choice(ports))+"/")

def address(addr):
    return str(addr).rjust(16, '0')


### View initial state

In [39]:
initial_utxo = requests.get(random_url() + "utxos/" + address(0)).content.decode('utf-8')
print(f"INITIAL UTXO\n {initial_utxo}")

INITIAL UTXO
 {
  "utxos": [{
    "txId": "0x00000000001",
    "utxoId": "00000000-0000-0000-0000-000000000000",
    "addr": "0000000000000000",
    "coins": "9223372036854775807"
  }]
}


### Send money between addresses handled by different shards


In [40]:
# from 0 to 1
requests.post(random_url() + "sendCoins", params={"to": address(1), "from": address(0), "coins": "100"})
# from 1 to 2
requests.post(random_url() + "sendCoins", params={"to": address(2), "from": address(1), "coins": "90"})
# from 2 to 3
requests.post(random_url() + "sendCoins", params={"to": address(3), "from": address(2), "coins": "80"})
# from 3 to 4
requests.post(random_url() + "sendCoins", params={"to": address(4), "from": address(3), "coins": "70"})
# from 4 to 1
requests.post(random_url() + "sendCoins", params={"to": address(1), "from": address(4), "coins": "60"})

<Response [200]>

### Get UTxOs after send money series

In [41]:
for addr in range(1, 5):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")


1 UTxOs

{
  "utxos": [{
    "txId": "0c88d434-7a73-456a-8e97-7866eafec899",
    "utxoId": "fb050ba2-2cac-3746-8f28-b8f52fa79756",
    "addr": "0000000000000001",
    "coins": "10"
  }, {
    "txId": "a5e13722-b44b-4784-8d62-b548b0962f43",
    "utxoId": "540049d8-bdeb-3955-9807-816bac6eb9f6",
    "addr": "0000000000000001",
    "coins": "60"
  }]
}

2 UTxOs

{
  "utxos": [{
    "txId": "054ec576-0e31-4997-ab80-5499fdccf4ed",
    "utxoId": "efd07a94-5320-3baf-8624-0a9e6513c39e",
    "addr": "0000000000000002",
    "coins": "10"
  }]
}

3 UTxOs

{
  "utxos": [{
    "txId": "1ef9512b-6f43-4637-a8f0-20880eed249e",
    "utxoId": "c719c251-e0b5-3d1d-8f16-9bee7e83e74f",
    "addr": "0000000000000003",
    "coins": "10"
  }]
}

4 UTxOs

{
  "utxos": [{
    "txId": "a5e13722-b44b-4784-8d62-b548b0962f43",
    "utxoId": "8d471940-c0ba-377e-89d8-de3ba3ec50d9",
    "addr": "0000000000000004",
    "coins": "10"
  }]
}


### Get ledger history for address 0

In [42]:
ledger = requests.get(random_url() + "ledger/" + address(0)).content.decode('utf-8')
print(f"Ledger for 0 \n{ledger}")

Ledger for 0 
This is the history for address 0000000000000000:
{
  "txs": [{
    "txId": "359a2568-def8-42d4-a963-2b50735afc27",
    "inputs": [{
      "txId": "0x00000000001",
      "utxoId": "00000000-0000-0000-0000-000000000000",
      "addr": "0000000000000000",
      "coins": "9223372036854775807"
    }],
    "outputs": [{
      "addr": "0000000000000001",
      "coins": "100"
    }, {
      "addr": "0000000000000000",
      "coins": "9223372036854775707"
    }],
    "timestamp": "2022-01-26T17:49:59.454Z"
  }]
}


### Get entire ledger history

In [43]:
ledger = requests.get(random_url() + "ledger/").content.decode('utf-8')
print(f"{ledger}")

This is the entire ledger history at the moment:
{
  "txs": [{
    "txId": "a5e13722-b44b-4784-8d62-b548b0962f43",
    "inputs": [{
      "txId": "1ef9512b-6f43-4637-a8f0-20880eed249e",
      "utxoId": "445819a2-d8aa-3f6e-81c4-fa0947cbbb21",
      "addr": "0000000000000004",
      "coins": "70"
    }],
    "outputs": [{
      "addr": "0000000000000001",
      "coins": "60"
    }, {
      "addr": "0000000000000004",
      "coins": "10"
    }],
    "timestamp": "2022-01-26T17:50:02.417Z"
  }, {
    "txId": "054ec576-0e31-4997-ab80-5499fdccf4ed",
    "inputs": [{
      "txId": "0c88d434-7a73-456a-8e97-7866eafec899",
      "utxoId": "9354ba62-7060-3a15-ae06-4084be9bed51",
      "addr": "0000000000000002",
      "coins": "90"
    }],
    "outputs": [{
      "addr": "0000000000000003",
      "coins": "80"
    }, {
      "addr": "0000000000000002",
      "coins": "10"
    }],
    "timestamp": "2022-01-26T17:50:01.650Z"
  }, {
    "txId": "1ef9512b-6f43-4637-a8f0-20880eed249e",
    "inputs": [

### Make a transaction

Transfer half of the coins for address 1 to address 0

In [44]:

def build_tx(src, dst):
    src = address(src)
    dst = address(dst)
    
    utxos_1 = json.loads(requests.get(random_url() + "utxos/" + src).content.decode('utf-8'))
    coins = sum([int(utxo['coins']) for utxo in utxos_1['utxos']]) 
    half_coins = coins // 2
    output1 = {"addr": dst, "coins": str(half_coins)}
    output2 = {"addr": src, "coins": str(coins-half_coins)}
    new_tx = {}
    new_tx['inputs'] = utxos_1['utxos']
    new_tx['outputs'] = output1,output2
    return new_tx


new_tx = json.dumps(build_tx(0, 1)).encode('utf-8')
res = requests.post(random_url() + "submitTx", data=new_tx, headers={'Content-Type': 'application/json'})
print(res.content.decode('utf-8'))

Submitted successfully with ID c6cca89a-92c3-44b6-b72b-e5fe559725f0 


### Print updated UTxOs for 0 and 1 after the transaction

In [45]:
for addr in range(0, 2):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")


0 UTxOs

{
  "utxos": [{
    "txId": "c6cca89a-92c3-44b6-b72b-e5fe559725f0",
    "utxoId": "4a5da852-d0da-3a18-98bc-f353637303ca",
    "addr": "0000000000000000",
    "coins": "4611686018427387854"
  }]
}

1 UTxOs

{
  "utxos": [{
    "txId": "0c88d434-7a73-456a-8e97-7866eafec899",
    "utxoId": "fb050ba2-2cac-3746-8f28-b8f52fa79756",
    "addr": "0000000000000001",
    "coins": "10"
  }, {
    "txId": "a5e13722-b44b-4784-8d62-b548b0962f43",
    "utxoId": "540049d8-bdeb-3955-9807-816bac6eb9f6",
    "addr": "0000000000000001",
    "coins": "60"
  }, {
    "txId": "c6cca89a-92c3-44b6-b72b-e5fe559725f0",
    "utxoId": "79b769b0-3c2e-32ab-ab02-92ea94575cb1",
    "addr": "0000000000000001",
    "coins": "4611686018427387853"
  }]
}


## Atomic List

In [46]:
def build_atomic():
    
    tx_list = list()
    
    for src, dst in zip([0, 1, 2, 3, 4], [1, 2, 3, 4, 0]):
        tx_list.append(build_tx(src, dst))
    
    return {"txs": tx_list}
    

In [47]:

atomic_tx_list = json.dumps(build_atomic()).encode('utf-8')

res = requests.post(random_url() + "submitAtomicTxList", data=atomic_tx_list, headers={'Content-Type': 'application/json'})
print(res.content.decode('utf-8'))


{
  "txIdsList": [{
    "txId": "Submitted successfully with ID 72f8d434-008e-45ec-bb86-d3b9b6f0982d "
  }, {
    "txId": "Submitted successfully with ID 2ab05113-1979-4b8d-8f0a-b09821aff17b "
  }, {
    "txId": "Submitted successfully with ID f0bac8a2-15c4-4db9-9b35-4eb7547c938c "
  }, {
    "txId": "Submitted successfully with ID 427c9215-002a-4572-9f95-1b884f2da498 "
  }, {
    "txId": "Submitted successfully with ID a130e924-5a1e-4483-bd84-05465b613059 "
  }]
}


In [48]:
for addr in range(0, 4):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")


0 UTxOs

{
  "utxos": [{
    "txId": "72f8d434-008e-45ec-bb86-d3b9b6f0982d",
    "utxoId": "11e2624e-03d2-3079-a76a-f2bace2e2568",
    "addr": "0000000000000000",
    "coins": "2305843009213693927"
  }, {
    "txId": "a130e924-5a1e-4483-bd84-05465b613059",
    "utxoId": "3e8f6089-ec4b-35fe-86ac-2ade739e03b0",
    "addr": "0000000000000000",
    "coins": "5"
  }]
}

1 UTxOs

{
  "utxos": [{
    "txId": "72f8d434-008e-45ec-bb86-d3b9b6f0982d",
    "utxoId": "f94a55b5-3ea1-39c2-a777-e9f553b3c4a6",
    "addr": "0000000000000001",
    "coins": "2305843009213693927"
  }, {
    "txId": "2ab05113-1979-4b8d-8f0a-b09821aff17b",
    "utxoId": "8e77242c-c897-3ffb-8d31-f7c840de4685",
    "addr": "0000000000000001",
    "coins": "2305843009213693962"
  }]
}

2 UTxOs

{
  "utxos": [{
    "txId": "2ab05113-1979-4b8d-8f0a-b09821aff17b",
    "utxoId": "30eda503-c4e7-347b-88aa-30ee934e22c6",
    "addr": "0000000000000002",
    "coins": "2305843009213693961"
  }, {
    "txId": "f0bac8a2-15c4-4db9-9b35-4eb